# Testing GPT-2 

In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
from datasets import load_dataset
import numpy as np
import evaluate

In [2]:
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", num_labels=2)
# default to left padding
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

In [3]:
dataset_test = load_dataset("csv", data_files="books_test.csv")

In [4]:
device = torch.device('cpu')
torch.set_num_threads(16)

In [5]:
model = GPT2ForSequenceClassification.from_pretrained("gpt2").to(device)

model.config.pad_token_id = model.config.eos_token_id
# resize model embedding to match new tokenizer
# model.resize_token_embeddings(len(tokenizer))


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
logits_list = []
data = dataset_test['train']['body']
batch_size = 10

for start in range(0, len(data), batch_size):
    print("Processing " + str(start))
    batch = data[start:start + batch_size]
    # tokenize all first?
    inputs = tokenizer(batch, truncation=True, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        logits_list.append(model(**inputs).logits)
logits = torch.cat(logits_list)


Processing 0
Processing 10
Processing 20
Processing 30
Processing 40
Processing 50
Processing 60
Processing 70
Processing 80
Processing 90


In [7]:
load_accuracy = evaluate.load("accuracy")
load_f1 = evaluate.load("f1")
labels = dataset_test['train']['label']

predictions = np.argmax(logits.data.cpu().numpy(), axis=-1)
accuracy = load_accuracy.compute(predictions=predictions, references=labels)
f1 = load_f1.compute(predictions=predictions, references=labels)
metrics = {"accuracy": accuracy, "f1": f1}

In [8]:
metrics

{'accuracy': {'accuracy': 0.4}, 'f1': {'f1': 0.3333333333333333}}

In [8]:
metrics

{'accuracy': {'accuracy': 0.53}, 'f1': {'f1': 0.624}}

In [11]:
metrics

{'accuracy': {'accuracy': 0.68}, 'f1': {'f1': 0.8048780487804879}}

In [9]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0])